In [2]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import json

In [3]:
def extract(result):
    day = result["unix_time"] // 60 // 60 // 24 - 19814
    if "total_originality" in result:
        return day, result["total_originality"]
    else:
        return day, float(result["accuracy"] > 0)

from collections import defaultdict

results = defaultdict(lambda: defaultdict(list))

for filename in glob("data/*_results.jsonl"):
    task = filename.split("/")[-1].split("_")[0]
    print(filename)
    with open(filename) as f:
        points = [extract(json.loads(line)) for line in f]
    for time, value in points:
        results[time][task].append(value)

data/AU_results.jsonl
data/RAT_results.jsonl
data/US_results.jsonl
data/PI_results.jsonl
data/IS_results.jsonl


In [7]:
days = sorted(list(results.keys()))
averaged = defaultdict(dict)
by_task = defaultdict(list)

for day in days:
    for task in results[day]:
        averaged[day][task] = np.mean(results[day][task])
        by_task[task].extend(results[day][task])

for day in days:
    for task in results[day]:
        averaged[day][task] = (np.mean(results[day][task]) - np.mean(by_task[task])) / np.std(by_task[task])

In [8]:
scores = dict()
for day in days:
    scores[day] = np.mean([averaged[day][task] for task in averaged[day] if task != "RAT"])

In [9]:
scores

{0: 0.0,
 2: -0.8016975932231286,
 3: 1.1310462256179872,
 6: -0.6592194637284163,
 7: -0.29859054564179943,
 9: 0.914776065954411}